---
title: ipympl + ipywidgets movie embedding
author: Georgios Varnavides
date: 2023/07/30
---

This is an example widget illustrating how to use ipympl and ipywidgets together.

In [52]:
#| label: app:ipympl_interactive_ipywidgets
# Widget combining ipympl + ipywidgets.

%matplotlib widget

from ipywidgets import interact, Play, Checkbox, Output, IntSlider, Dropdown, IntRangeSlider, Layout, HBox, VBox
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import display
import imageio as iio
import numpy as np

# Import video frames
vid = iio.get_reader("data/45grains_compressed.mp4")
num_frames = vid.count_frames()
dpi = 70

array = vid.get_data(0)
array = array[:,:,0] # grayscale
array = 255 - array # reverse

plt.ioff()

hist_range_plot = (-4,4)
hist_range_init = (-1,2)
hist_num_bins = 200

# Histogram visualization
def compute_histogram_values(array):
    int_mean = np.mean(array)
    int_std = np.sqrt(np.mean((array - int_mean)**2))
    
    int_min = int_mean + hist_range_plot[0] * int_std
    int_max = int_mean + hist_range_plot[1] * int_std
    
    init_min = int_mean + hist_range_init[0] * int_std
    init_max = int_mean + hist_range_init[1] * int_std
    
    int_ranges = (int_mean, int_std, int_min, int_max,init_min,init_max)
    
    hist_bins = np.linspace(
        int_ranges[2],
        int_ranges[3],
        hist_num_bins+1,
        endpoint=True)
    hist_data, _ = np.histogram(
        array.ravel(),
        bins=hist_bins,
    )
    hist_data = hist_data.astype('float')
    hist_data /= np.max(hist_data)
    
    hist_bins = hist_bins[:-1] + (hist_bins[1] - hist_bins[0])/2
    
    return hist_bins, hist_data, int_ranges

hist_bins, hist_data, int_ranges = compute_histogram_values(array)
fig_hist, ax_hist = plt.subplots(figsize=(175/dpi, 150/dpi), dpi=dpi)

h_hist = ax_hist.fill_between(
    hist_bins,
    hist_data,
    color = (0, 0.7, 1.0, 1.0),
)

h_vlines =ax_hist.vlines(
    int_ranges[4:6],
    ymin = 0,
    ymax = 1.1,
    color = 'k',
)

ax_hist.set_xlim((int_ranges[2], int_ranges[3]));
ax_hist.set_ylim((0, 1.1));
ax_hist.set(yticks=[])
ax_hist.set(yticklabels=[])

fig_hist.canvas.toolbar_visible = False
fig_hist.canvas.header_visible = False
fig_hist.canvas.footer_visible = False
fig_hist.canvas.resizable = False
fig_hist.tight_layout()


# Main visualization
fig, ax = plt.subplots(figsize=(400/dpi, 400/dpi), dpi=dpi)

fig.canvas.resizable = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.toolbar_visible = True
fig.canvas.layout.width = '400px'
fig.canvas.toolbar_position = 'bottom'

ax.axis('off')

im=ax.imshow(array,cmap='Greys',vmin=int_ranges[4],vmax=int_ranges[5])
divider = make_axes_locatable(ax)
ax_cb = divider.append_axes("right", size="5%", pad="2.5%")
fig.add_axes(ax_cb)
fig.colorbar(im, cax=ax_cb)
fig.tight_layout()

scalebar_plot=ax.plot((20,70),(484-20,484-20),lw=5,color='k')
scalebar_text=ax.text((70+20)/2,484-20-10,"2 nm",color='k',ha='center')

def display_frame(change):

    # main
    index = change['new']
    array = vid.get_data(index)[:,:,0] # grayscale image
    array = 255 - array # reverse
    im.set_data(array)
    fig.canvas.draw_idle()

    # hist
    hist_bins, hist_data, int_ranges = compute_histogram_values(array)
    v_x = np.hstack([hist_bins[0],hist_bins,hist_bins[-1],hist_bins[::-1],hist_bins[0]])
    v_y = np.hstack([hist_data[0],np.zeros_like(hist_data),0,hist_data[::-1],hist_data[0]])
    vertices = np.vstack([v_x,v_y]).T
    codes = np.array([1]+(2*len(hist_bins)+1)*[2]+[79]).astype('uint8')
    
    path = h_hist.get_paths()[0]
    path.vertices = vertices
    path.codes = codes
    fig_hist.canvas.draw_idle()
    
    return None

def toggle_scalebar(change):
    scalebar = change['new']
    scalebar_plot[0].set_visible(scalebar)
    scalebar_text.set_visible(scalebar)
    fig.canvas.draw_idle()
    return None

def update_colormap(change):
    cmap = change['new']
    im.set_cmap(cmap)
    fig.canvas.draw_idle()
    return None

def update_vlines(change):
    min, max = change['new']
    p = h_vlines.get_segments()
    p = np.array([
        [
            [min, 0],
            [min, 1.1],
        ],
        [
            [max, 0],
            [max, 1.1],
        ]
    ])
    h_vlines.set_segments(p)
    fig_hist.canvas.draw_idle()
    
    im.set_clim([min,max])
    fig.canvas.draw_idle()
    return None

index_widget = Play(value=0,min=0,max=num_frames-1,step=1,interval=50,layout=Layout(width='175px'))
index_widget.observe(display_frame,names='value')

scalebar_widget =Checkbox(value=True,description="Show scale bar",indent=False,layout=Layout(width='175px'))
scalebar_widget.observe(toggle_scalebar,names='value')

sequential_cmaps = [
    'viridis', 'plasma', 'inferno', 'magma', 'cividis','turbo',
    'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
    'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
    'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn'
]

cmap_widget =Dropdown(options=sequential_cmaps,value='Greys',description="Colormap",indent=False,layout=Layout(width='175px'))
cmap_widget.observe(update_colormap,names='value')

histogram_range_slider = IntRangeSlider(
    value=int_ranges[4:6],
    min=int_ranges[2],
    max=int_ranges[3],
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    indent=True,
    layout=Layout(width='175px')
)
histogram_range_slider.observe(update_vlines,names='value')

visualization_layout = Layout(
    display='flex',
    flex_flow='row',
    align_items='center',
    width='600px'
)

histogram_box_layout = Layout(
    display='flex',
    flex_flow='column',
    align_items='center',
    width='200px'
)
histogram_vbox = VBox([fig_hist.canvas,histogram_range_slider],layout=histogram_box_layout)

controls_vbox = VBox([histogram_vbox,index_widget,cmap_widget,scalebar_widget],layout=histogram_box_layout)

display(
    HBox(
        [
            fig.canvas,
            controls_vbox
        ],
        layout=visualization_layout
        )
)